In [18]:
import re
import itertools 
import logging
import sys

In [19]:
logging.basicConfig(format='%(asctime)s,%(msecs)d %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    datefmt='%Y-%m-%d:%H:%M:%S',
    level=logging.INFO,filemode = 'w')
logging.disable(sys.maxsize)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler('logging.log')
logger.addHandler(file_handler)

In [20]:
def remove_space(string):
    new_str = ''
    for item in string.split():
        new_str += item
    return new_str

def bcopy(olist):
    new_list = []
    for bs in olist:
        new_list.append(bs.copy())
    return new_list

def countp(string):
# input a bngl format reaction, compare the substanes('+') number of left side(left_count) and right side(right_count)
# return (left_count-right_count)
    left = string.split('-')[0]
    right = string.split('-')[1]
# left and right are two sides of the input reaction
    left_count = 0
    right_count = 0
    for char in left:
        if char == '+':
            left_count += 1
        else:
            pass
    for char in right:
        if char == '+':
            right_count += 1
        else:
            pass
    return (left_count - right_count)

def is_substance(string):
# Determine if the input string is a substance in the form X(x,y,...).Y(x,y,...). ...
    for char in string:
        if char == '(':
            return True
        else:
            pass
    return False

def makename(name):
# Convert a name like X(x,y,...).Y(x,y,...). ... to X().Y(). ...
    sp1 = name.split('(')                     # X(x,y,...).Y(x,y,...). ... .split('(') = X, x,y,...).Y, ...
    n0=''
    n0 += sp1[0]                              # n0 = 'X'
    if len(sp1) == 2:                         # if there is only one component
        pass
    else:
        for i,item in enumerate(sp1):
            if i == 0:
                pass
            else:
                n0 += item.split(')')[-1]    # n0 = n0 + '.Y'
    return n0

def reaction_gen(reaction):
# Input a reaction_rule object, generate a bngl format reaction string                                 
    reactantlist = reaction.get_reactants()  
    productlist = reaction.get_products()
# Read the reactans and products lists
    left, right = '', ''
    for rxt in reactantlist:
        left += rxt.get_prname() + ' + '
    for rxt in productlist:
        right += rxt.get_prname() + ' + '
    rxn = left[0:-3] + ' ' + reaction.get_dir() + ' ' + right[0:-3] + ' ' + reaction.get_eq()
    return rxn

def if_contained(sub_aa, sub_bb):
# input two list, usually from a reactant object.get_component()
    sub_a = sub_aa.copy()
    sub_b = sub_bb.copy()
    for a in sub_a:
        re = False
        for b in sub_b:
            if a == b:
                re = True              # When find the same element in sub_b as in sub_a
                break              
            else:
                pass
        if re == False:
            return False
        else:
            sub_b.remove(a)            # remove the element from b
    return True                        # If the loop is done, every element in sub_a can find the same a in sub_b, return true

def if_same_sub(sub1,sub2):
# input two list, usually from a reactant object.get_component()
# if the elements in two list are exactly the same, return True
        s1 = sub1.get_name().split('.')
        s2 = sub2.get_name().split('.')
        if len(s1) != len(s2):
            return False
        else: 
            for item in s1:
                for jtem in s2:
                    if item == jtem:
                        s1.remove(item)
                        s2.remove(jtem)
        if s1 == [] and s2 == []:
            return True
        else:
            return False
        


    
'''def component_replacement(product1, reactant1,reactant0):
# Replace the component x in reactant1 with reactant2y if-elif checks. In the future should an unexpected 'corner case' raise its ugly head this bit of documentation may very well help to find the 
    new_product = ''
    to_be_altered = []
    for cp_r0 in reactant0.split('.'):
        for cp_p1 in product1.split('.'):
            if cp_r0.split('(')[0] == cp_p1.split('(')[0]:
                to_be_altered.append(cp_p1)
            else:
                new_product += '.' + cp_p1
    new_product = new_product[1:-1] + '.' + reactant1
    for cp_ta in to_be_altered:
        for cp_np in new_product.split('.'):
            if cp_ta.split('(') == cp_np.split('('):
                sites_ta = cp_ta.split('(')[1].split(')')[0].split(',')
                sites_np = np_ta.split('(')[1].split(')')[0].split(',')
                for site_ta in sites_ta:
                    for site_np in sites_np:
                        if (site_ta.split('!')[0] ==  site_np.split('!')[0]) and (len(site_ta) > len(site_np)):
                            new_product = new_product.split(cp_np.split('(')[0]) + site_ta + new_product.split(site_np)[1]
                        else:
                            pass
    return new_product'''

def find_bonded_components(reactant1):
    logger.info('Starting to find bonded components of {}'.format(reactant1))
    if len(reactant1.split('!')) < 2:
        return ([],[],[])
    reactant1 += ' '
    sp1 = reactant1.split('!')
    sp2 = sp1.copy()
    out = []
    bond_name = []
    site_name = []
    for i,item1 in enumerate(sp1):
        for j,item2 in enumerate(sp2):
            if i >= j:
                continue
            elif item1.split(')')[0].split(',')[0] == item2.split(')')[0].split(',')[0]:
                #print(134,item1,'  ',item2)
                #print(135,reactant1.split(item1)[0].split('.'),reactant1.split(item1)[0].split('.')[-1].split('('))
                site_name.append([sp1[i-1].split('(')[-1].split(',')[-1],sp2[j-1].split('(')[-1].split(',')[-1]])
                bond_name.append(item1.split(')')[0].split(',')[0])
                item1 = '!' + item1
                item2 = '!' + item2
                out.append([reactant1.split(item1)[0].split('.')[-1].split('(')[0],reactant1.split(item2)[-2].split('.')[-1].split('(')[0]])
                logging.info('two items are {} and {},found a bond between{}'.format(item1,item2,out[-1]))
            else:
                pass
            
    for i,o1 in enumerate(out):
        for o2 in out:
            #print(0,o1,o2)
            #print(out)
            if (o1 != o2) and (set(o1) == set(o2)):
                del site_name[i]
                del bond_name[i]
                out.remove(o1)
            else:
                #print(1,o1,o2)
                pass
    return (out,bond_name,site_name)



def bond_update(r0_list):
    r_list = []
    for r0 in r0_list:
        r_list.append(r0.rcopy())
    o_n = 0
    if len(r_list) == 1:
        return r_list
    else:
        for i,r in enumerate(r_list):
            if len(r.get_prname().split('!')) == 1:
                continue
            elif i == 0:
                o_n += len(r.get_bond_info()[1])
            else:
                for sites in r.get_bindingsites():
                    if sites == []:
                        continue
                    for site in sites:
                        if site.is_empty():
                            continue
                        else:
                            site.bind(str(int(site.get_bond())+o_n))
                r.set_prname()
                #print(r.get_bond_info())
                o_n += len(r.get_bond_info()[1])
                #print(o_n)
            #print(r.get_prname())
    return r_list


def pr_name_scaler(pr_name1):
    #return pr_name1
    if len(pr_name1.split('!')) < 2:
        return pr_name1
    else:
        alist = pr_name1.split('.')
        alist.sort()
        pr_name1 = ''
        for item in alist:
            pr_name1 += '.'+item
        pr_name1 = pr_name1[1:]
        #print(pr_name1)
        resip = []
        old_bonds = []
        conslist0 = pr_name1.split('!')
        conslist1 = pr_name1.split('!')
        for i in range(int((len(conslist0)-1)/2)):
            resip.append(i+1)
        for i,item in enumerate(conslist0):
            if i == 0:
                continue
            elif i == 1:
                old_bonds.append([conslist1[1][0],i])
                conslist1[1] = '1' + conslist1[1][1:]
                del resip[0]
            else:
                jug = True
                #print(229,item,old_bonds)
                for bond in old_bonds:
                    if item[0] == bond[0]:
                        conslist1[i] = conslist1[bond[1]][0] + conslist1[i][1:]
                        jug = False
                        break
                if jug == True:
                    old_bonds.append([conslist1[i][0],i])
                    conslist1[i] = str(resip[0]) + conslist1[i][1:]
                    del resip[0]
    new_pr = ''
    for item in conslist1:
        new_pr += '!' + item
    return new_pr[1:]

def find_new_bonds(reactants_list,product_list):
    r_bonds = []
    r_components = []
    for rxt in reactants_list:
        bonded = find_bonded_components(rxt.get_prname())
        r_bonds += bonded[2]
        r_components +=bonded[0]
    p_bonds = []
    p_components = []
    for pr in product_list:
        bonded = find_bonded_components(pr.get_prname())
        p_bonds += bonded[2]
        p_components +=bonded[0]
    for i,p_component in enumerate(p_components):
        for j,r_component in enumerate(r_components):
            if set(r_component) == set(p_component):
                del r_components[j]
                del p_components[i]
                del r_bonds[j]
                del p_bonds[i]
                continue
    if r_components != []:
        logger.info('Bonds {} cannot be found in products'.format(r_components))
    return p_components,p_bonds
                
    
def is_available(reactants, newbond):
    bond = newbond[1][0]
    bonded_comp = newbond[0][0]
    b0 = False
    b1 = False
    for rxt in reactants:
        for i,comp in enumerate(rxt.get_component()):
            if comp == bonded_comp[0]:
                for site in rxt.get_bindingsites()[i]:
                    if site.get_site() == bond[0] and site.is_empty():
                        b0 = True
                        break
            elif comp == bonded_comp[1]:
                for site in rxt.get_bindingsites()[i]:
                    if site.get_site() == bond[1] and site.is_empty():
                        b1 = True
                        break
    if b0 and b1:
        return True
    else:
        print(reactants[0].get_prname(),newbond)
        return False
    
def mut_rxt(rxts_list):
    pr_list = []
    for rxts in rxts_list:
        pr_list.append([])
        for rxt in rxts:
            pr_list[-1].append(rxt.get_prname())
    inters = pr_list[0]
    for rxts in pr_list:
        inters = set(inters).intersection(rxts)
    return list(inters)

In [21]:
def product_gen(rxts_list,new_bonds,p_complex,moleculelist):
    # temperate product generator
    logger.info('New bonds to form are {}'.format(new_bonds[0]))
    temp_prolist = []
    temp_rxt = []
    looping_rxt = mut_rxt(rxts_list)
    rxntls = list(itertools.product(*rxts_list))
    for loop in looping_rxt:
        #print(9,loop)
        rxntls.append([reactant(loop)])
    logger.info('Product will generate from {}'.format(rxntls))
    for rxntl in rxntls:
        temp_pro = []
        bond_list = new_bonds[0].copy()
        site_list = new_bonds[1].copy()
        logger.info('Product_gen processing, reactants are {}'.format(list(rxntl)))
        pro = ''
        '''for rxnt in list(rxntl):
            while True:
                #print(rxnt)
                if re.findall(' ',rxnt) == []:
                    break
                else:
                    rxnt = rxnt.replace(' ','')'''
        #logger.info('{} and {} are going to be updated'.format(list(rxntl)[0].get_prname(),list(rxntl)[1].get_prname()))
        rxntl_up = bond_update(list(rxntl))
        if is_available(rxntl_up,new_bonds) == False:
            '''if len(rxntl_up == 1):
                print(29)'''
            continue
        #elif
        else:
            temp_rxt.append(list(rxntl))
            #logger.info('--------------Product is going to generate from {} and {}'.format(rxntl_up[0].get_prname(),rxntl_up[1].get_prname()))
            for rxnt in rxntl_up:
                logger.info('Product_gen processing, reactant added is {}'.format(rxnt.get_prname()))
                pro += '.'+rxnt.get_prname()
                #print(26,rxnt.get_prname())
                bond_list += find_bonded_components(rxnt.get_prname())[0]
                site_list += find_bonded_components(rxnt.get_prname())[2]
                #print(31,site_list)
            logger.info('Product {} is generating'.format(pro[1:]))
            product = reactant(pro[1:])
            if product.complex_check(p_complex) == False:
                '''if len(rxntl_up) == 1:
                    print(46,pro[1:])'''
                del temp_rxt[-1]
                #print(36,len(temp_rxt),len(temp_prolist))
                continue
            logger.info('Product {} is generated, with bonded sites {} in {}'.format(product.get_prname(),site_list,product.get_siteslist()))
            logger.info('Product {} is generated, adding bond are between {}'.format(product.get_prname(),bond_list))
            product.set_bonds_info([bond_list,[],site_list])
            #product.
            temp_pro.append(product)
            #print(133,temp_prolist)
            temp_prolist.append(temp_pro)
    return [temp_rxt,temp_prolist]

'''def find_new_bonds(reaction1):
    rxts = reaction1.get_reactants()
    pts = reaction1.get_products()
    

def is_occupied(rxts_list,new_bond):'''
    

'def find_new_bonds(reaction1):\n    rxts = reaction1.get_reactants()\n    pts = reaction1.get_products()\n    \n\ndef is_occupied(rxts_list,new_bond):'

In [22]:
class reactant:
    
    def __init__(self, substance):
        self.name = makename(substance)       # Name in a form like A.B.C. ...
        self.bindingsites = []                # bindingsites stores a list of lists of binding_site objects, [[ob1,ob2],[],[ob3], ...]
        self.component = []                   # a list of components in string format
        self.pr_name = remove_space(substance)            # th full name with all binding sites appear, like A(a,b!1,c,d,e).B(a!1,g,h)
        self.bonded_components = []
        self.bond_names = []
        self.correlated_sites =[]
        self.component_init(substance)

    def component_init(self,substance):
        for cn,c in enumerate(substance.split('(')):
            if cn == 0:                                                          #the first component in a substance before '.'
                self.component.append(c)
            elif cn == len(substance.split('('))-1:                              # the last bindsites after '.'          
                if re.findall('.+?(?=\))',c) == []:
                    self.bindingsites.append('')   
                else:
                    inp = re.findall('.+?(?=\))',c)[0].split(',')                # What inside a '.(*)'
                    tempsite = []
                    for site in inp:
                        tempsite.append(binding_site(site.split('!')[0]))        # define binding sites
                        if len(site.split('!')) > 1:
                            tempsite[-1].bind(site.split('!')[1])                # name the bond
                        else:
                            pass
                    self.bindingsites.append(tempsite)
            else:
                self.component.append(c.split('.')[-1])
                if re.findall('.+?(?=\))',c) == []:
                    self.bindingsites.append('')                
                else:
                    inp = re.findall('.+?(?=\))',c)[0].split(',')
                    tempsite = []
                    for site in inp:
                        tempsite.append(binding_site(site.split('!')[0]))
                        if len(site.split('!')) > 1:
                            logger.debug('{}, There is a bond {}'.format(self.name, site))
                            tempsite[-1].bind(site.split('!')[1])
                        else:
                            pass
                    self.bindingsites.append(tempsite)
            
        self.bonded_components,self.bond_names, self.correlated_sites = find_bonded_components(substance)
    
    def rcopy(self):
        return reactant(self.get_prname())
    
    def set_bonds_info(self,info_list):
        self.bonded_components = info_list[0]
        self.bond_names = []
        for i in range(len(info_list[0])):
            self.bond_names.append(str(i+1))
        #print(53,self.get_bindingsites(),self.get_component())
        
        for j,bonded_comp in enumerate(info_list[0]):
            logger.info('bonded components are {}'.format(bonded_comp))
            p1 = self.c_match(bonded_comp[0],1)
            p2 = self.c_match(bonded_comp[1],1)
            #print(62,info_list,self.get_prname())
            s1 = self.c_match(info_list[2][j][0],2,p1)
            s2 = self.c_match(info_list[2][j][1],2,p2)
            logger.info('binding sites are{},{},{}'.format(self.get_siteslist(),self.get_bindingsites()[p1],s1))
            #logger.info('p1 = {}, p2 ={}'.format(p1,p2))
            self.get_bindingsites()[p1][s1].bind(self.bond_names[j])
            self.get_bindingsites()[p2][s2].bind(self.bond_names[j])
        self.set_prname()
            
        
    
    def c_match(self,cp,m,n=0):
        if m ==1:
            for i,comp in enumerate(self.get_component()):
                if cp == comp:
                    return i
        elif m == 2:
            for i, bs in enumerate(self.get_bindingsites()[n]):
                #print(cp,bs.get_site())
                if cp == bs.get_site():
                    return i
            logger.info('{} has no match! in {}'.format(cp,self.get_siteslist()))
    
    def is_subrxt(self,parent):             
    # determine if self is the a form of parent, parent is also a reactant object
        if if_contained(self.get_component(), parent.get_component()) == False:
            return False                     # first check if the components are contained in parent
        else:
            p_comps = parent.get_component().copy()         # p_comps stores a copy of parent's component
            p_bonds = parent.get_siteslist().copy()         # p_bonds stores a copy of parent's sites, a!1
            comps = self.get_component().copy()
            bonds = self.get_siteslist().copy()             # These two are copies of own components and sites(a!1)
            mar = False
            for j, comp in enumerate(comps):
                for i, p_comp in enumerate(p_comps):
                    logger.debug('Self = {}({}), Parent = {}({}) '.format(comp,bonds[j],p_comp,p_bonds[i]))
                    if comp == p_comp:
                        if if_contained(bonds[j], p_bonds[i]):
                            logger.debug('component {} matches with component{}'.format(comp, p_comp))
                            mar = True
                            break
                        else:
                            pass
                    else:
                        pass
                if mar == True:
                    p_comps.remove(p_comp) # Everytime there is a match, delete the element in parent
                    del p_bonds[i]
                else:
                    logger.debug('component {} has no matche in parent'.format(comp))
                    return False
            return True                    # if every own component can find a match in parent, return True
        
    def complex_check(self,parent):             
        if if_contained(self.get_component(), parent.get_component()) == False:
            return False  
        else:
            return True
        
   
        

                    
    
    def is_same_molecule(self,molecule1):
        s_n1 = set(pr_name_scaler(self.get_prname()).split('.'))
        m_n1 = set(pr_name_scaler(molecule1.get_prname()).split('.'))
        '''
        s_n = set(self.get_prname().split('.'))
        m_n = set(molecule1.get_prname().split('.'))
        if (s_n1 == m_n1) != (s_n == m_n):
            print(130)
            print(s_n1,m_n1,s_n1 == m_n1)
            print(s_n,m_n,s_n == m_n)
            '''
        if s_n1 != m_n1:
            return False
        else:
            #print(s_n,m_n)
            '''for i,cp1 in enumerate(s_cp):
                for j,cp2 in enumerate(m_cp):
                    if cp1 == cp2:
                        del s_cp[i]
                        del m_cp[j]
            if s_cp == [] and m_cp == []:
                return True
            else:'''
            return True
            
    def is_existed(self,molecule_list1):
        for mol in molecule_list1:
            if self.is_same_molecule(mol):
                return True
            else:
                pass
                '''print(self.get_prname(),'does not exist in')
                for mmmmm in moleculelist:
                    print(mmmmm.get_prname())
                print()'''
        return False
        
    def replace_full_bs(self,begin_mol0):        
        # This function will regard the bonds not mentioned to be empty, and update bindingsites accordingly
        
        begin_mol = []
        for mol in begin_mol0:
            begin_mol.append(mol.rcopy())
        is_in_begin_molecules = False
        for i,self_c in enumerate(self.component):
            for mol in begin_mol:       # Take begining molecules as reference
                if self_c == mol.get_component()[0]:   # Assume the begining molecule only have one component
                    is_in_begin_molecules = True
                    logger.info('{} match the molecule'.format(mol.get_prname()))
                    complete_sites = bcopy(mol.get_bindingsites())  # To replace the [] in self.bindingsites
                    self_bts = bcopy(self.get_bindingsites())      

                    if len(self.get_bindingsites()[i]) == len(complete_sites[0]):  # See if self is already in full binding
                        logger.debug('{} is already in full format'.format(self.get_prname()))               # sites format
                        break
                    else:
                        for j,ss in enumerate(complete_sites[0]): # Assume the begining molecules only have one component
                            for self_ss in self_bts[i]:
                                if self_ss.get_site() == ss.get_site():
                                    logger.info('the molecule being replaced is {}'.format(self.get_prname()))
                                    logger.info('the {}th binding site of {} is empty, and will be replaced to {}'.format(j,self.get_name(),self_ss.get_site_bond()))
                                    loggerlist = []
                                    for logcom in complete_sites[0]:
                                        loggerlist.append(logcom.get_site_bond())
                                    logger.info('complete_sites is {}'.format(loggerlist))
                                    complete_sites[0][j] = self_ss
                                    self_bts[i].remove(self_ss)
                                    break
                                else:
                                    pass
                        self.bindingsites[i] = complete_sites[0]

                else:
                    pass
                
            if is_in_begin_molecules == False:
                logger.info('The {} of {} is not in begining molecule list, process terminated'.format(self_c,self.get_name()))
        self.set_prname()
        logger.debug('Old pr_name is {}'.format(self.get_prname()))
        self.set_prname()
        logger.debug('New pr_name is {}'.format(self.get_prname()))
    
    def set_prname(self):
    # Set pr_name based on name and binding sites
        p = ''
        elist = self.get_siteslist()
        #print(149,elist,self.get_component())
        for i,cp in enumerate(self.get_component()):
            p += cp + '('
            site = ''
            for ci in elist[i]:
                site += ci + ','
            p += site[0:-1] + ').'
        self.pr_name = p[0:-1]
    
    def get_siteslist(self):
    # Get a string version of own binding sites
        eplist = []
        for bs in self.get_bindingsites():
            if bs == []:
                eplist.append([])
            else:
                eplist.append(bs.copy())
        
        for i,site in enumerate(eplist):
            if site == []:
                pass
            else:
                for j,si in enumerate(site):
                   # print(157,si,site)
                    eplist[i][j] = si.get_site_bond()
        return eplist
    
    def get_component(self):
        return self.component
    
    def get_bindingsites(self):
        return self.bindingsites
    
    def get_name(self):
        return self.name
    
    def get_bond_info(self): 
        return ([self.bonded_components, self.bond_names])
    
    def get_prname(self):
        return self.pr_name
    
                    
class binding_site:
# Define a class to describe every binding site in a reactant class
    def __init__(self, name):
        self.name = name
        self.ifempty = True
        self.bond_name = ''
    def bind(self, bindn):
        self.ifempty = False
        self.bond_name = bindn
    
    def bindto(self, substance):
        self.bindingto = substance3
        
    def get_site(self):
        return self.name
        
    def is_empty(self):
        return self.ifempty
        
    def get_bond(self):
        return self.bond_name
    
    def get_site_bond(self):
        if self.ifempty:
            return self.name
        else:
            #print(231,self.bond_name)
            return (self.name + '!' +self.bond_name)
        
        
class reaction_rule:
    
    def __init__(self, name):
        self.name = name
        self.reactants = []
        self.products = []
        self.bindingsite = []
        self.eq_con = ''
        self.direction = ''
        self.new_bonds_info = []
        
    def set_reactants(self,left):
        self.reactants.append(left)
        
    def set_products(self,right):
        self.products.append(right)
        
    def set_eqcon(self, eqcon):
        self.eq_con = eqcon
        
    def set_direc(self,direc):
        self.direction = direc
            
    def set_newbonds(self):
        self.new_bonds_info = find_new_bonds()
    
    def get_reactants(self):
        return self.reactants
    
    def get_newbonds(self):
        return self.new_bonds_info
    
    def get_products(self):
        return self.products
    
    def get_eq(self):
        return self.eq_con
    
    def get_dir(self):
        return self.direction
    
    def find_new_bonds(self):
        r_bonds, r_names = [],[]
        p_bonds, p_names = [],[]
        ite = []
        for r in self.reactants:
            r_bonds.append(r.get_bond_info[0])
            r_names.append(r.get_bond_info[1])
        for p in self.products:
            p_bonds.append(p.get_bond_info[0])
            p_names.append(p.get_bond_info[1])
        for r in r_bonds:
            for j,p in enumerate(p_bonds):
                if set(r) == set(p):
                    ite.append(j)
                    continue
                else:
                    pass
        new_bonds = p_bonds.copy()
        new_names = p_names.copy()
        for j in ite:
            del new_bonds[j]
            del new_names[j]
        return ([new_bonds,new_names])
    
    def is_same_reaction(self,reaction1):
        if self.get_eq() != reaction1.get_eq() or self.get_dir() != reaction1.get_dir():
            return False
        else:
            for item in self.get_reactants():
                if item.is_existed(reaction1.get_reactants()) == False:
                    return False
        return True
                    
    
    def is_existed(self,rule_list):
        for rxn in rule_list:
            if self.is_same_reaction(rxn) == True:
                return True
        return False
        
            

In [33]:
class reaction(reaction_rule):
    def __init__(self, name):
        reaction_rule.__init__(self,name)

In [37]:
#read begin_molecule_type from a .bngl format file
moleculelist=[]
myfile1=open('./input/ABCDm.txt','r')
for line in myfile1.readlines():     # Skip comments
    if line.split()[0][0] == '#':
        pass
    else:
        line = line.split('#')[0].split('\t')[-1].split('\n')[0]     #lines start with (\t) and end with (\n)
        moleculelist.append(reactant(line))                          # Append reactant class reactants to moleculelist
        moleculelist[-1].component_init(line)                        # Initialize the object with name(line)
myfile1.close()
for m in moleculelist:
    logger.info('{}{} is added to beginging molecule list'.format(m.get_name(),m.get_component()))
begin_molecule = []
for mol in moleculelist:     # Molecule list stores all the molecules generated, begin_molecule only the begining ones
    begin_molecule.append(mol.rcopy())

In [38]:
# read reaction rules
reactionrules = []                         # Store the rules, 
passlist=[]                                # Store the rules not able to handle in current version
myfile2 = open('./input/ABCDloop_r-Copy4.txt','r')
rulenum = 1
for line in myfile2.readlines():
    ktemp=[]
    if line.split()[0][0] == '#':
        continue
    else:
        line = line.split('#')[0]           # Skip the comments
        if len(line.split('~')) > 1:
            passlist.append(line)            # Not able to handle reactions with '~' yet, it will be transfered to output directly 
            continue
        elif countp(line)<0: 
            passlist.append(line)            # If the raction is a breakup reaction, no new substance will appear, also directly to output
            continue
        else:
            reactionrules.append(reaction_rule(rulenum))
            rulenum += 1 
            reactionrules[-1].set_eqcon(line.split(')')[-1])      # Extract reaction constant(s), should the component after the last ')'
            reactionrules[-1].set_direc(re.findall('.-.',line)[0])  # Extract the direction of the rule (<-> or ->)
            leftr = line.split('-')[0].split()
            rightr = line.split('-')[1].split()                     # Two sides of the rule are processed separately
            for item in leftr:
                if is_substance(item):
                    pass
                else:
                    leftr.remove(item)
            for item in rightr:
                if is_substance(item):
                    pass
                else:
                    rightr.remove(item)
            for item in leftr:
                reactionrules[-1].set_reactants(reactant(item))
            for item in rightr:
                #print(item)
                reactionrules[-1].set_products(reactant(item))
 # Initialize the newly appended rule by setting reactants and products
myfile2.close()
'''vvv = reactant('CHC1(a2!1).ENT(a!1)')
vvv.replace_full_bs(begin_molecule)
print(vvv.get_prname())
for mmmmm in moleculelist:
    print(mmmmm.get_prname())'''

"vvv = reactant('CHC1(a2!1).ENT(a!1)')\nvvv.replace_full_bs(begin_molecule)\nprint(vvv.get_prname())\nfor mmmmm in moleculelist:\n    print(mmmmm.get_prname())"

In [39]:

#generate rule-based reactions
#reaction_gen(reactionrules[0])
#p_complex = reactant('CHC1(a1!1,a2,a3,b1!4,b2,b3,c1,c2,c3).CLC1(a!2,b!1).ENT(a!2,b,lb!3).EDE1(a!3,c!4)')
p_complex = reactant('A(b!1,c!2,d!3).B(a!1,c!4,d!5).C(a!2,b!4,d!6).D(a!3,b!5,c!6)')
reactions_list = []
reactions_ite = 1
n_repeat = 0
ite_w = 1
while (n_repeat != len(reactionrules)):
    #print(ite_w,'n_repeat',n_repeat,len(reactionrules))
    ite_w += 1
    n_repeat = 0
    for i,rrule in enumerate(reactionrules):
        #print(7,rrule.get_products()[0].get_prname())
        #print(i)
        repeat = False
        temp_rxnt_list = []
        looping_list = []
        for rxnt in rrule.get_reactants(): # [A(b).B(a),B(b),...]
            temp_dlist = []
            for molec in moleculelist:
                #print(18,molec.get_bindingsites()[0])
                if rxnt.is_subrxt(molec):
                    #print(15)
                    temp_dlist.append(molec)
                    #temp_dlist.append(molec.get_prname())
            if temp_dlist == []:
                print('error')
                break
            else:
                temp_rxnt_list.append(temp_dlist)
        #print(33,rrule.get_products()[0].get_prname())
        logger.info('reactants list is {}'.format(temp_rxnt_list))
        rrule.get_products()[0].replace_full_bs(begin_molecule)
        n_bonds = find_new_bonds(rrule.get_reactants(),rrule.get_products())
        magic_list = product_gen(temp_rxnt_list,n_bonds,p_complex,moleculelist)
        #print(34,magic_list[0],magic_list[1])

        sub_repeat = False
        for magic_i,magic_rs in enumerate(magic_list[0]):

            reactions_list.append(reaction(reactions_ite))
            reactions_ite += 1
            for magic_r in magic_rs:
                magic_temp1 = reactant(magic_r.get_prname())
                #print(magic_temp1.get_component(),len(magic_list[1]),magic_i)
                reactions_list[-1].set_reactants(magic_temp1)
            for magic_p in magic_list[1][magic_i]:

                magic_temp2 = reactant(pr_name_scaler(magic_p.get_prname()))
                reactions_list[-1].set_products(magic_temp2)
                reactions_list[-1].set_eqcon(rrule.get_eq())
                reactions_list[-1].set_direc(rrule.get_dir())
                if magic_p.is_existed(moleculelist) == True and reactions_list[-1].is_existed(reactions_list[:-1]) == True:
                    del reactions_list[-1]
                    logger.info('Product {} does not make sense'.format(magic_p.get_prname()))
                else: 
                    sub_repeat = True
                    '''#print(45,magic_p)    
                    for mmmmm in moleculelist:
                        print(mmmmm.get_prname())
                    print(magic_temp2.if_existed(moleculelist),magic_temp2.get_prname())
                    print()'''
                    moleculelist.append(magic_temp2)

                    '''if len(reactions_list)>0:
                        print(reaction_gen(reactions_list[-1]))'''
                    
        if sub_repeat == False:
            logger.info('Rule {} has no new reactions'.format(i))
            #print('repeat',i)
            n_repeat += 1
        #if len(reactions_list)>0:
            #print(reaction_gen(reactions_list[-1]))
myfile3 = open('output.txt','w')
print('reactions are:')
for r222 in reactions_list:
    myfile3.write((reaction_gen(r222).split('\n')[0]+'\n'))               
#print(temp_rxnt_list)
myfile3.close()

reactions are:


In [15]:
len(moleculelist)

82

In [ ]:
# assume that when more than three components are forming a complex, self-looping is faster than connecting to a new component 